# INTPBD3 ASSIGNMENT

My website of choice for the assignment is:
- [Airbnb](https://www.airbnb.co.uk/) https://www.airbnb.co.uk/

## About Airbnb

Airbnb is a short term rental marketplace that connects people looking for short term stays with property owners who are willing to have them. Owners post details of their property such as photos, specifications, and rules. A user will register onto the platform by sharing their personal and payment information with Airbnb. Once registered, users can look for a place they like, and once they find it, they will book for a specific date and pay. 

As a startup founded in 2008, they knew they needed scalability in their database design because being global was one of their targets. Flexibility is also important for such a company so they can introduce more services along the way such as Airbnb experiences.

## Relational Design

The first thing to consider is usually which kind of database to use. Engineers will have to choose between a relational and a NoSQL design. In the case of Airbnb, there is data that fits in well with a relational design, which we will discuss later, and some that is uniquely suited to a NoSQL type of database.

Once an engineer has settled on a relational database design based on the kind of data the company will be storing. The first question becomes which relational/SQL database to use.

As a startup where money is a problem, an open source SQL database is the way to go. We eliminate two major players in the closed source SQL database space: Microsoft SQL Server and Oracle Database.

The open source relational database space has two key players: MySQL and PostgresSQL. Here is a summary of key differences as discussed by [AWS blog](https://aws.amazon.com/compare/the-difference-between-mysql-vs-postgresql/).

|MySQL|PostgresSQL|
|:--------------------|:---------------------------|
|No Multiversion Concurrency Control(MVCC)|Has MVCC|
|Supports B-tree and R-tree indexing|Indexing includes:trees, expression indexes, partial indexes, and hash indexes|
|Pure relational database|Object-relational database|
|Supports views|Views with more advanced options|
|Can call stored procedures in SQL|Can call stored procedures in other languages|
|Can only use AFTER and BEFORE triggers for SQL INSERT, UPDATE, and DELETE statements|Supports the INSTEAD OF trigger, so you can run complex SQL statements using functions|

PostgresSQL seems to have overall better features but a startup trying to move fast and break things may want to go with the option that is easiest to get started and recruit from; that is MySQL.

For our use case, it doesn't matter which of the two the organization chooses because we are designing the website at a high level and we are not yet concerned with features like the kinds of indexing supported. Let's say they do settle on MySQL, how do they proceed?

### Entities

The evident entities that Airbnb has that would fit in a tabular structure include:
1. Users (both customers and owners)
    - Customers and owners share a lot of attributes. It is also possible for an owner to be a tenant while on vacation.
2. Bookings
3. Payments

Consider each column in the table as representing an ER diagram. They are connected by signs showing  their cardinality.

|Users||Bookings||Payments|
|:----|:-----|:-----|:-----|:-----|
|First_Name|1 - - - - - m|Property_ID|1 - - - - - 1|User_ID|
|Last_Name||Date_of_Booking||Payment_Method|
|Email||User_ID||Date_of_Payment|
|Is_Owner||Receipt_ID||Amount|
|Is_Customer||End_Date_of_Booking||Booking_ID|
|ID_Number||Booking_ID||Payment_Instrument_Expiration_Date|
|Phone_Number||||Billing_Postcode|
|Postal_Address||||Payment_ID|
|DOB|||||
|Profile_Photo_URL|||||
|User_ID|||||
||||||

- The `Users` and `Bookings` table have a cardinality of one-to-many; a user can make many bookings in their lifetime.
- `Users` and `Payments` have a one-to-many relationship too because by going on many trips, a user will make many payments.
- `Bookings` and `Payments` tables have a one-to-one relationship; each booking has one payment and a payment is meant for one booking.

Each entity has a set of attributes that we will use when describing the table's schema.

### `Users` Schema

|Field|Data Type||
|:---|:---|:---|
|First_Name|CHAR(50)|
|Last_Name|CHAR(50)|
|Email|CHAR(70)|
|Is_Owner|BOOL|
|Is_Customer|BOOL|
|ID_Number|CHAR(50)|
|Phone_Number|CHAR(50)|
|Postal_Address|CHAR(200)|
|Post_Code|CHAR(20)|
|DOB|DATETIME|
|Password|CHAR(50)|
|Profile_Photo_URL|CHAR(200)|
|User_ID|CHAR(20)|PRIMARY KEY|

### `Bookings` Schema

|Field|Data Type||
|:---|:---|:---|
|Booking_ID|CHAR(20)|PRIMARY KEY|
|Date_of_Booking|DATETIME|
|End_Date_of_Booking|DATETIME|
|User_ID|CHAR(20)|FOREIGN KEY|
|Payment_ID|CHAR(20)|FOREIGN KEY|
|Property_ID|CHAR(20)|FOREIGN KEY|

### `Payments` Schema

|Field|Data Type||
|:---|:---|:---|
|Payment_Method|CHAR(30)||
|Date_of_Payment|DATETIME||
|Amount|DECIMAL(8,2)||
|Billing_Postcode|CHAR(10)||
|Payment_ID|CHAR(20)|PRIMARY KEY|
|Payment_Instrument_Expiration_Date|DATETIME||
|Booking_ID|CHAR(20)|FOREIGN KEY|
|User_ID|CHAR(20)|FOREIGN KEY|


With the schemas displayed above we have an idea of how the tables are represented in the database. We need queries to retrieve data from the database and some of the expected queries are:
1. When the user attemps to log into the website, we have to verify their password. We retrieve the password hash stored in their row by finding their email.
   - `SELECT Password FROM Users WHERE Email = "useremail@example.com";`

2. Once the user has successfully logged, they can navigate to their dashboard where they can access their personal details. We retrieve all the users personal details by using the query:
   - `SELECT * FROM Users WHERE Email = "useremail@example.com";` or `SELECT * FROM Users WHERE User_ID = "unique_user_id";`

3. We may also want to show a user the upcoming bookings they have. We filter out past bookings.
   - `SELECT * FROM Bookings WHERE User_ID = "unique_user_id" and Date_of_Travel > (SELECT CURDATE());`
  
4. Lastly, a user may want to see dates when they booked and the payments between a specific time interval..
   - `SELECT Date_of_Booking, Amount FROM Bookings INNER JOIN Payments on Bookings.Booking_ID = Payments.Booking_ID WHERE Date_of_Booking > "example-start-date" AND End_Date_of_Booking <= "example-end-date" AND User_ID = "unique_user_id"`

## Non-Relational Design / NoSQL Design

Properties in Airbnb are the entities that require the most flexibility in their schema. Each property has multiple reviews about it that are unique to it. A NoSQL database like MongoDB allows us to store this information in an array of objects. Each property also has images that are unique to it. Instead of having a table in MySQL with image urls and a foreign key pointing to the property, it is far easier in MongoDB to have an array of strings, where the strings are the url or an array of objects if we wish to store more than urls. 

Booking destinations have different features. For us to store the information that one property has a swimming pool while another doesn't in MySQL, we would need columns of a Boolean data type. We would also have to create a new column everytime the company decides to track a new feature. This would make the property table very wide over time.

For us to store the kind of data required to describe a property in SQL databases, we would need multiple tables. This in turn means more complicated queries or multiple queries. With MongoDB we get all of a property's information from a single query.

### `Properties` Model

```
reviews_model = {
    comment: String,
    date: Date
}

photos_model = {
    url: String,
    mimeType: String,
    imagePosition: Integer
}

properties_model = {
    title: String,
    write_up: String,
    location: String,
    owner_ID: String, # User_ID
    reviews: [reviews_model],
    photos: [photos_model],
    features: [String]
}
```

To define the property model, we first create the `reviews_model` and the `photos_model` because they are sub-collections of the `properties_model`.

With MongoDB, you don't have to specify the number of characters per key. It simply takes strings and doesn't care about the length. It also generates its own objectID value which is guaranteed to be unique so we don't have to worry about what the primary key is and how we'll make it unique.

### `Users` Model

```
users_model = {
    first_name: String,
    last_name: String,
    email: String,
    is_owner: Boolean,
    is_customer: Boolean,
    ID_number: String,
    phone_number: String,	
    postal_address:	String,	
    post_code: String,
    DOB: Date,
    password: String,
    profile_photo_URL: String,	
}
```

### `Bookings` Model

NoSQL entities - Property, Reviews. Property has reviews, features, photos, availability

Considerations
- What to do about booking dates to avoid them clashing
- 

## References

- MySQL Datatypes - https://dev.mysql.com/doc/refman/8.3/en/data-types.html
- MongoDB Datatypes - https://www.mongodb.com/docs/manual/reference/bson-types/